# 5G NR CP Autocorrelation Detection Statistic

Computes the normalized cyclic-prefix autocorrelation

$$\hat{\rho} = \frac{|\hat{R}(N_u)|}{\hat{R}(0)}, \quad
\hat{R}(\tau) = \frac{1}{L} \sum_{n=0}^{L-1} x[n]\, x^*[n+\tau]$$

over non-overlapping windows of $L = 2^{17}$ samples from the srsRAN recording.

For 5G NR with 15 kHz SCS at 7.68 MSPS:
- $N_u = 512$ (useful symbol length)
- $N_{\mathrm{cp,normal}} = 36$, $N_{\mathrm{cp,extended}} = 40$ (symbols 0 and 7 per slot)
- Theoretical value: $\rho_{\mathrm{true}} = \Sigma N_{\mathrm{cp}} / L_{\mathrm{slot}} = 512/7680 = 1/15 \approx 0.0667$ (high SNR)

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (10, 4)

In [ ]:
# Load SigMF recording (ci32_le: interleaved int32 I/Q)
data_path = 'catkira/1876954_7680KSPS_srsRAN_Project_gnb_short.sigmf-data'
raw = np.fromfile(data_path, dtype='<i4')
x = raw[0::2].astype('float32') + 1j * raw[1::2].astype('float32')
print(f'Loaded {len(x)} complex samples ({len(x)/7.68e6*1e3:.1f} ms at 7.68 MSPS)')

In [ ]:
# Parameters for 5G NR 15 kHz SCS at 7.68 MSPS
fs     = 7.68e6
N_u    = 512          # useful symbol length: fs / 15000
L      = 2**17        # window size: 131072 samples ≈ 17.1 ms

# Theoretical ρ at high SNR:
# 2 extended-CP symbols (40 samples) + 12 normal-CP symbols (36 samples) per slot
# total CP per slot = 2*40 + 12*36 = 512; total samples per slot = 7680
rho_theory = 512 / 7680
print(f'Theoretical ρ (high SNR) = {rho_theory:.4f}  (= 1/15 = {1/15:.4f})')
print(f'Window length L = {L} samples = {L/fs*1e3:.2f} ms')
print(f'Number of non-overlapping windows: {len(x) // L}')

In [ ]:
# Compute ρ̂ for each non-overlapping window
n_windows = len(x) // L
rho = np.empty(n_windows)

for i in range(n_windows):
    seg = x[i*L : (i+1)*L]
    R_lag  = np.mean(seg[:-N_u] * np.conj(seg[N_u:]))   # complex lag-N_u autocorrelation
    R_zero = np.mean(np.abs(seg)**2)                     # power estimate
    rho[i] = np.abs(R_lag) / R_zero

print(f'ρ̂ mean = {rho.mean():.5f}   std = {rho.std():.5f}')
print(f'ρ̂ min  = {rho.min():.5f}   max = {rho.max():.5f}')
print(f'Theoretical ρ = {rho_theory:.5f}')

In [ ]:
# Histogram of the detection statistic
fig, ax = plt.subplots()

ax.hist(rho, bins=12, density=True, alpha=0.7, label=r'$\hat{\rho}$ per window')
ax.axvline(rho_theory, color='C1', linestyle='--', linewidth=1.5,
           label=fr'$\rho_{{\mathrm{{theory}}}} = 1/15 = {rho_theory:.4f}$')
ax.axvline(rho.mean(), color='C2', linestyle='-', linewidth=1.5,
           label=fr'sample mean = {rho.mean():.4f}')

ax.set_xlabel(r'$\hat{\rho} = |\hat{R}(N_u)| / \hat{R}(0)$')
ax.set_ylabel('Density')
ax.set_title(fr'CP autocorrelation statistic, $L = 2^{{17}}$ samples, $N_u = {N_u}$')
ax.legend()
plt.tight_layout()

## Notes

- With $L = 2^{17} = 131072$ samples, each window spans ~17 ms (~1.7 slots).  
  The estimator averages over $L - N_u \approx 130560$ lag pairs, giving a tight distribution.
- The sample mean should be close to $\rho_{\mathrm{theory}} = 1/15 \approx 0.0667$;  
  any small deviation reflects finite-sample bias and SNR < ∞.
- Under H0 (AWGN, no OFDM), $\hat{R}(N_u)$ is the average of $L - N_u$ i.i.d. zero-mean  
  complex Gaussian terms, so $\hat{\rho}$ follows a **Rayleigh** distribution with scale  
  $\sigma_{\mathrm{H0}} \approx 1 / \sqrt{L - N_u} \approx 0.0028$, far below the 5G value.

## Robustness to carrier frequency offset (CFO)

A CFO of $\Delta f$ Hz multiplies the baseband signal by $e^{j 2\pi \Delta f n / f_s}$.
Substituting into the lag product:

$$x[n]\,x^*[n+N_u]
= s[n]\,s^*[n+N_u]\cdot e^{j 2\pi \Delta f n/f_s}\cdot e^{-j 2\pi \Delta f (n+N_u)/f_s}
= s[n]\,s^*[n+N_u]\cdot \underbrace{e^{-j 2\pi \Delta f N_u/f_s}}_{\text{constant}}$$

The CFO factor is **constant** (independent of $n$), so it factors out of the sum:

$$\hat{R}_{\Delta f}(N_u) = e^{-j 2\pi \Delta f N_u / f_s}\cdot \hat{R}_0(N_u)$$

Taking the magnitude removes the phase factor entirely:

$$\hat{\rho}_{\Delta f} = \frac{|\hat{R}_{\Delta f}(N_u)|}{\hat{R}(0)} = \frac{|\hat{R}_0(N_u)|}{\hat{R}(0)} = \hat{\rho}_0$$

**The detection statistic $\hat{\rho}$ is exactly invariant to CFO**, for any $\Delta f$.
The CFO is not lost — it is encoded in the **phase** of $\hat{R}(N_u)$:

$$\widehat{\Delta f} = -\frac{f_s}{2\pi N_u}\,\angle\hat{R}(N_u)
\qquad\text{(unambiguous range: }|\Delta f| < f_s/(2 N_u) = 7.5\text{ kHz)}$$

This is the same ±1 subcarrier ambiguity as in Schmidl & Cox.

In [ ]:
# Numerical verification: apply CFOs spanning many subcarrier spacings,
# confirm ρ̂ is unchanged while phase rotates predictably.

seg = x[:L]
n   = np.arange(L, dtype='float64')

delta_fs_table = [0, 1, 100, 1000, 7500, 50_000, 500_000, 3_000_000]

print(f'{"Δf (Hz)":>12}   {"ρ̂":>10}   {"∠R̂(Nᵤ) (°)":>14}   {"predicted (°)":>14}')
print('-' * 58)
for df in delta_fs_table:
    rot     = seg * np.exp(1j * 2 * np.pi * df * n / fs)
    R_lag   = np.mean(rot[:-N_u] * np.conj(rot[N_u:]))
    R_zero  = np.mean(np.abs(rot)**2)
    rho_val = np.abs(R_lag) / R_zero
    meas    = np.degrees(np.angle(R_lag))
    pred    = np.degrees((-2 * np.pi * df * N_u / fs + np.pi) % (2*np.pi) - np.pi)
    print(f'{df:>12,}   {rho_val:>10.6f}   {meas:>14.2f}   {pred:>14.2f}')

In [ ]:
# Sweep CFO across the full Nyquist range and plot ρ̂ (flat) and ∠R̂(Nᵤ) (sawtooth)

delta_fs_sweep = np.linspace(-fs/2, fs/2, 600)

rho_sweep   = np.empty(len(delta_fs_sweep))
phase_sweep = np.empty(len(delta_fs_sweep))

for i, df in enumerate(delta_fs_sweep):
    rot           = seg * np.exp(1j * 2 * np.pi * df * n / fs)
    R_lag         = np.mean(rot[:-N_u] * np.conj(rot[N_u:]))
    R_zero        = np.mean(np.abs(rot)**2)
    rho_sweep[i]  = np.abs(R_lag) / R_zero
    phase_sweep[i] = np.degrees(np.angle(R_lag))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(delta_fs_sweep / 1e3, rho_sweep)
ax1.axhline(rho_theory, color='C1', linestyle='--', label=r'$\rho_{\mathrm{theory}}$')
ax1.set_xlabel('CFO  (kHz)')
ax1.set_ylabel(r'$\hat{\rho}$')
ax1.set_title(r'$\hat{\rho}$ is flat — detection is CFO-blind')
ax1.legend()
ax1.set_ylim(0, None)

predicted_phase = np.degrees((-2*np.pi*delta_fs_sweep*N_u/fs + np.pi) % (2*np.pi) - np.pi)
ax2.plot(delta_fs_sweep / 1e3, phase_sweep, label=r'measured $\angle\hat{R}(N_u)$')
ax2.plot(delta_fs_sweep / 1e3, predicted_phase, '--', alpha=0.6,
         label=r'$-2\pi\Delta f N_u/f_s$ (wrapped)')
ax2.set_xlabel('CFO  (kHz)')
ax2.set_ylabel(r'$\angle\hat{R}(N_u)$  (deg)')
ax2.set_title(fr'Phase encodes CFO: unambiguous ±{fs/2/N_u/1e3:.1f} kHz = ±1 subcarrier')
ax2.legend()

plt.tight_layout()

### What the plots show

**Left**: $\hat{\rho}$ is completely flat across the entire Nyquist range (±3.84 MHz), equal to the no-CFO value to numerical precision for all tested offsets.

**Right**: $\angle\hat{R}(N_u)$ is a sawtooth — linear in $\Delta f$ and wrapping every $f_s/N_u = 15$ kHz (one subcarrier spacing). Within each wrap it directly estimates the CFO.

### Where robustness breaks down

| Effect | Impact on $\hat{\rho}$ |
|--------|------------------------|
| CFO (any size) | **None** — exact invariance proven above |
| SFO (−3.2 ppm in this recording) | Negligible: lag error grows to ≲0.4 samples over $L$, causing < 0.1% attenuation |
| Phase noise | Decorrelates CP from its copy; reduces $\hat{\rho}$ proportionally to accumulated phase-noise variance over $N_u$ samples (~67 µs at 15 kHz SCS) |
| Multipath within CP length | Unaffected: the CP is long enough to absorb inter-symbol interference, so the copy property is preserved |
| Signal shifted partially out of ADC bandwidth | Effective SNR loss → $\hat{\rho}$ falls toward zero at extreme offsets near ±$f_s/2$; not visible here because the high-SNR signal remains flat to the band edge |